# Extract Evaluation Positions for Next-Move Prediction

Dataset format: `(FEN, last_move, next_move_candidates, correct_outputs)`

Each record contains **two separate groups** of illegal moves as lists of `{"uci": str, "type": str}` dicts:
- `illegal_category` -- category-specific illegals (pin-breaking, king-to-attacked, castling-while-in-check, etc.)
- `illegal_general` -- general distractors (backward pawn, friendly fire, blocked sliding, wrong geometry, etc.)

Both are merged into `next_move_candidates_uci`; `correct_outputs_uci` = all legal moves.

**Position categories:**
1. **En passant** -- en passant is legal; distractors: `ep_fake_diagonal` (diagonal to empty, no adjacent pawn) + `ep_wrong_pawn` (adjacent enemy pawn that didn't just push)
2. **Check evasion (single)** -- in check by one piece; distractors: `king_to_attacked` + `castling_in_check`
3. **Double check** -- two checkers; only king moves legal; distractors: `non_king_double_check` + `king_to_attacked` + `castling_in_check`
4. **Illegal king moves** -- not in check; `king_to_attacked` + `castling_through_attacked`
5. **Pin** -- own piece pinned to own king; distractors: `pin_breaking`
6. **Promotion** -- pawn can promote; distractors: `promo_push_blocked` + `promo_capture_empty`
7. **Vanilla** -- random positions with no special tag; only general distractors

In [ ]:
import sys
import json
import random
from pathlib import Path
from collections import defaultdict, Counter
from typing import Optional, List, Set, Dict, Tuple

import chess
import chess.pgn

# Make sure local modules are importable
if str(Path('.').resolve()) not in sys.path:
    sys.path.insert(0, str(Path('.').resolve()))

from legal_moves import get_phase, iter_games, CASTLE_INFO
from legal_move_puzzles import (
    detect_en_passant, build_en_passant_illegals,
    analyze_check, build_check_candidates,
    detect_double_check, build_double_check_illegals,
    detect_illegal_king_moves, build_illegal_king_illegals,
    detect_pin, build_pin_illegals,
    detect_promotion, build_promotion_illegals,
    generate_general_distractors,
    make_row, extract_all,
)

# ── Config knobs ─────────────────────────────────────────────────────────────
PGN_PATH = Path('data/lichess_db_standard_rated_2013-01.pgn')
OUT_PATH = Path('data/eval_positions_preview.jsonl')
MAX_GAMES = 50                  # games to scan
NUM_GENERAL_DISTRACTORS = 5     # general illegal moves per position
NUM_VANILLA_POSITIONS = 100     # how many vanilla (no-tag) positions to sample
SEED = 50

rng = random.Random(SEED)

print("Imports and config loaded.")

In [ ]:
# Helpers, detectors, and builders are now in legal_moves.py and legal_move_puzzles.py.
# Quick smoke test:
b = chess.Board()
b.push_san("e4")
b.push_san("e5")
b.push_san("Nf3")
distractors = generate_general_distractors(b, set(m.uci() for m in b.legal_moves), rng, num_target=8)
print(f"Smoke test: {len(distractors)} distractors from position after 1.e4 e5 2.Nf3:")
for uci, dtype in distractors:
    print(f"  {uci:6s}  ({dtype})")

In [ ]:
# ── Run extraction ───────────────────────────────────────────────────────────
rows = extract_all(
    str(PGN_PATH), MAX_GAMES,
    num_general_distractors=NUM_GENERAL_DISTRACTORS,
    num_vanilla_positions=NUM_VANILLA_POSITIONS,
    rng=rng,
)

In [ ]:
# ── Summary statistics ───────────────────────────────────────────────────────
from collections import Counter

tag_counter = Counter()
phase_counter = Counter()
illegal_type_counter = Counter()
castling_count = 0
for r in rows:
    for t in r["tags"]:
        tag_counter[t] += 1
    phase_counter[r["phase"]] += 1
    if r.get("num_illegal_castling", 0) > 0:
        castling_count += 1
    for d in r.get("illegal_category", []) + r.get("illegal_general", []):
        illegal_type_counter[d["type"]] += 1

print("=== Tag distribution ===")
for tag, cnt in tag_counter.most_common():
    print(f"  {tag:20s}: {cnt}")

print(f"\n=== Phase distribution ===")
for phase, cnt in phase_counter.most_common():
    print(f"  {phase:20s}: {cnt}")

print(f"\n=== Illegal move type distribution ===")
for itype, cnt in illegal_type_counter.most_common():
    print(f"  {itype:30s}: {cnt}")

print(f"\n=== Candidate stats ===")
num_cands = [r["num_candidates"] for r in rows]
num_cat = [r["num_illegal_category"] for r in rows]
num_gen = [r["num_illegal_general"] for r in rows]
print(f"  Avg candidates:            {sum(num_cands)/len(num_cands):.1f}")
print(f"  Avg category illegals:     {sum(num_cat)/len(num_cat):.1f}")
print(f"  Avg general illegals:      {sum(num_gen)/len(num_gen):.1f}")
print(f"  Max category illegals:     {max(num_cat)}")
print(f"  Max general illegals:      {max(num_gen)}")
print(f"  Positions w/ cat illegals: {sum(1 for x in num_cat if x > 0)} / {len(rows)}")
print(f"  Positions w/ illegal castling: {castling_count}")

In [ ]:
# ── Inspect examples per category ────────────────────────────────────────────

def show_example(row, idx=None):
    """Print text details for one example."""
    prefix = f"[{idx}] " if idx is not None else ""
    print(f"{prefix}Tags: {row['tags']}  |  Phase: {row['phase']}  |  Ply: {row['ply']}")
    print(f"  FEN:        {row['fen']}")
    print(f"  Last move:  {row['last_move_uci']}")
    print(f"  Game move:  {row['game_move_uci']}")
    print(f"  Candidates: {row['num_candidates']}  "
          f"(legal={row['num_correct']}, cat_illegal={row['num_illegal_category']}, "
          f"gen_illegal={row['num_illegal_general']})")

    # Legal moves
    print(f"  Legal moves: {sorted(row['correct_outputs_uci'])}")

    # Category-specific illegals (list of dicts with uci + type)
    cat_ill = row.get("illegal_category", [])
    if cat_ill:
        print(f"  Category illegals:")
        for d in cat_ill:
            print(f"    {d['uci']:10s}  ({d['type']})")

    # General illegals (list of dicts with uci + type)
    gen_ill = row.get("illegal_general", [])
    if gen_ill:
        print(f"  General illegals:")
        for d in gen_ill:
            print(f"    {d['uci']:10s}  ({d['type']})")

    # Category-specific info
    if "en_passant" in row["tags"]:
        print(f"  EP moves: {row.get('ep_moves_uci', [])}")
    if "check" in row["tags"]:
        print(f"  Check evasions: king_moves={row.get('check_king_moves')}, "
              f"captures={row.get('check_captures')}, blocks={row.get('check_blocks')}, "
              f"illegal_king={row.get('check_illegal_king')}, "
              f"illegal_castling={row.get('check_illegal_castling', 0)}")
    if "double_check" in row["tags"]:
        print(f"  Double check: checkers={row.get('checker_squares')} "
              f"({row.get('checker_pieces')}), "
              f"legal_king={row.get('num_legal_king_moves')}")
    if "illegal_king" in row["tags"]:
        ic = row.get("num_illegal_castling", 0)
        if ic:
            print(f"  Illegal castling: {row.get('illegal_castling_uci', [])}")
    if "pin" in row["tags"]:
        print(f"  Pinned: {row.get('pinned_details', [])}")
    if "promotion" in row["tags"]:
        print(f"  Promotion moves: {row.get('promo_moves_uci', [])}")
    print()

# Show examples per tag
for tag in ["en_passant", "check", "double_check", "illegal_king", "pin", "promotion", "vanilla"]:
    examples = [r for r in rows if tag in r["tags"]]
    print(f"{'='*60}")
    print(f"  {tag.upper()} -- {len(examples)} positions")
    print(f"{'='*60}")
    for r in examples[:2]:
        show_example(r)

In [ ]:
# ── Visualize a few boards (SVG) ─────────────────────────────────────────────
import chess.svg
from IPython.display import SVG, display, HTML

def show_board(row, title=""):
    board = chess.Board(row["fen"])
    last = chess.Move.from_uci(row["last_move_uci"])

    cat_ill = [d["uci"] for d in row.get("illegal_category", [])]
    gen_ill = [d["uci"] for d in row.get("illegal_general", [])]

    # Arrows: yellow=last move, red=category illegals, orange=general illegals
    arrows = [chess.svg.Arrow(last.from_square, last.to_square, color="#888800")]
    for u in cat_ill[:4]:
        m = chess.Move.from_uci(u)
        arrows.append(chess.svg.Arrow(m.from_square, m.to_square, color="#cc0000"))
    for u in gen_ill[:3]:
        m = chess.Move.from_uci(u)
        arrows.append(chess.svg.Arrow(m.from_square, m.to_square, color="#cc8800"))

    svg = chess.svg.board(board, arrows=arrows, size=350)
    label = (f"<b>{title}</b><br/>Tags: {row['tags']} | Phase: {row['phase']}<br/>"
             f"Cat illegals (red): {len(cat_ill)} | Gen illegals (orange): {len(gen_ill)}")
    display(HTML(f"<div style='display:inline-block; margin:10px'>{label}<br/>{svg}</div>"))

# Show one example per category
for tag in ["en_passant", "check", "double_check", "illegal_king", "pin", "promotion", "vanilla"]:
    examples = [r for r in rows if tag in r["tags"]]
    if examples:
        show_board(examples[0], title=tag.upper())

In [15]:
# ── Save to JSONL ────────────────────────────────────────────────────────────
with OUT_PATH.open("w", encoding="utf-8") as f:
    for row in rows:
        f.write(json.dumps(row) + "\n")

print(f"Saved {len(rows)} positions to {OUT_PATH}")
print(f"Fields per record: {sorted(rows[0].keys())}")

Saved 1176 positions to data/eval_positions_preview.jsonl
Fields per record: ['correct_outputs_uci', 'fen', 'game_id', 'game_move_uci', 'general_distractor_types', 'illegal_category_uci', 'illegal_general_uci', 'last_move_uci', 'next_move_candidates_uci', 'num_candidates', 'num_correct', 'num_illegal_castling', 'num_illegal_category', 'num_illegal_general', 'num_illegal_king_moves', 'num_legal_king_moves', 'phase', 'ply', 'tags']


## Sanity checks

Verify the extracted data:
1. All `correct_outputs_uci` are actually legal moves in the FEN
2. All illegal distractors are actually illegal
3. En passant moves are among the legal moves
4. Pinned piece moves in distractors are truly illegal

In [ ]:
# ── Sanity checks ────────────────────────────────────────────────────────────

errors = []
for i, row in enumerate(rows):
    board = chess.Board(row["fen"])
    legal_ucis = set(m.uci() for m in board.legal_moves)

    # Check 1: all correct outputs are legal
    for m in row["correct_outputs_uci"]:
        if m not in legal_ucis:
            errors.append(f"Row {i}: correct move {m} is NOT legal in {row['fen']}")

    # Check 2: correct = legal (should be the same set)
    if set(row["correct_outputs_uci"]) != legal_ucis:
        missing = legal_ucis - set(row["correct_outputs_uci"])
        extra = set(row["correct_outputs_uci"]) - legal_ucis
        if missing:
            errors.append(f"Row {i}: legal moves missing from correct: {missing}")
        if extra:
            errors.append(f"Row {i}: non-legal moves in correct: {extra}")

    # Check 3: all category illegals are actually illegal
    for d in row.get("illegal_category", []):
        if d["uci"] in legal_ucis:
            errors.append(f"Row {i}: category illegal {d['uci']} ({d['type']}) is actually LEGAL")

    # Check 4: all general illegals are actually illegal
    for d in row.get("illegal_general", []):
        if d["uci"] in legal_ucis:
            errors.append(f"Row {i}: general illegal {d['uci']} ({d['type']}) is actually LEGAL")

    # Check 5: no overlap between category and general illegals
    cat_set = set(d["uci"] for d in row.get("illegal_category", []))
    gen_set = set(d["uci"] for d in row.get("illegal_general", []))
    overlap = cat_set & gen_set
    if overlap:
        errors.append(f"Row {i}: overlap between cat and gen illegals: {overlap}")

    # Check 6: candidates = legal + cat_illegal + gen_illegal
    expected = set(row["correct_outputs_uci"]) | cat_set | gen_set
    actual = set(row["next_move_candidates_uci"])
    if expected != actual:
        errors.append(f"Row {i}: candidates mismatch (expected {len(expected)}, got {len(actual)})")

    # Check 7: en passant specific
    if "en_passant" in row["tags"]:
        for ep_uci in row.get("ep_moves_uci", []):
            if ep_uci not in legal_ucis:
                errors.append(f"Row {i}: ep move {ep_uci} not legal")

    # Check 8: every illegal entry has both 'uci' and 'type' keys
    for d in row.get("illegal_category", []) + row.get("illegal_general", []):
        if "uci" not in d or "type" not in d:
            errors.append(f"Row {i}: illegal entry missing uci/type keys: {d}")

if errors:
    print(f"ERRORS ({len(errors)}):")
    for e in errors[:20]:
        print(f"  {e}")
else:
    print(f"All {len(rows)} positions passed sanity checks!")

In [17]:
# ── Tag co-occurrence ────────────────────────────────────────────────────────
all_tags = ["en_passant", "check", "double_check", "illegal_king", "pin", "promotion", "vanilla"]
cooc = defaultdict(int)
for r in rows:
    ts = sorted(set(r["tags"]))
    key = " + ".join(ts) if len(ts) > 1 else ts[0]
    cooc[key] += 1

print("Tag combinations:")
for combo, cnt in sorted(cooc.items(), key=lambda x: -x[1]):
    print(f"  {combo:45s}: {cnt}")

Tag combinations:
  illegal_king                                 : 800
  check                                        : 110
  pin                                          : 103
  vanilla                                      : 100
  illegal_king + pin                           : 31
  illegal_king + promotion                     : 12
  promotion                                    : 7
  en_passant                                   : 4
  check + pin                                  : 3
  double_check                                 : 3
  en_passant + illegal_king                    : 1
  check + promotion                            : 1
  en_passant + pin                             : 1


In [ ]:
# ── Example JSON record ──────────────────────────────────────────────────────
import pprint

# Pick an example with both category and general illegals
ex = next((r for r in rows if r["num_illegal_category"] > 0 and r["num_illegal_general"] > 0), rows[0])
clean = {
    "fen": ex["fen"],
    "last_move_uci": ex["last_move_uci"],
    "next_move_candidates_uci": f"[{ex['num_candidates']} items]",
    "correct_outputs_uci": f"[{ex['num_correct']} items]",
    "illegal_category": ex["illegal_category"][:5],
    "illegal_general": ex["illegal_general"],
    "tags": ex["tags"],
    "phase": ex["phase"],
    "num_candidates": ex["num_candidates"],
    "num_correct": ex["num_correct"],
    "num_illegal_category": ex["num_illegal_category"],
    "num_illegal_general": ex["num_illegal_general"],
}
print("Example record (lists truncated for display):")
pprint.pprint(clean)

## Interactive Browser

`browse(rows)` -- step through examples with Enter.  
Commands: `Enter`=next, `p`=prev, `q`=quit, number=jump, `t TAG`=filter by tag.

Arrow colors: **yellow**=last move, **red**=category illegals, **orange**=general illegals.

In [ ]:
import chess.svg
from IPython.display import SVG, display, HTML, clear_output


def show_full(row, idx, total):
    """Display board SVG + full text details for one example."""
    clear_output(wait=True)
    board = chess.Board(row["fen"])
    last = chess.Move.from_uci(row["last_move_uci"])

    cat_ill = [d["uci"] for d in row.get("illegal_category", [])]
    gen_ill = [d["uci"] for d in row.get("illegal_general", [])]

    # Arrows: yellow=last move, red=cat illegals, orange=gen illegals
    arrows = [chess.svg.Arrow(last.from_square, last.to_square, color="#888800")]
    for u in cat_ill[:5]:
        m = chess.Move.from_uci(u)
        arrows.append(chess.svg.Arrow(m.from_square, m.to_square, color="#cc0000"))
    for u in gen_ill[:3]:
        m = chess.Move.from_uci(u)
        arrows.append(chess.svg.Arrow(m.from_square, m.to_square, color="#cc8800"))

    svg = chess.svg.board(board, arrows=arrows, size=400)
    display(HTML(svg))

    print(f"── Example {idx}/{total-1} ──")
    show_example(row, idx)

    # Illegal type breakdown
    all_illegals = row.get("illegal_category", []) + row.get("illegal_general", [])
    if all_illegals:
        print(f"  Illegal type breakdown:")
        from collections import Counter
        for dtype, cnt in Counter(d["type"] for d in all_illegals).most_common():
            print(f"    {dtype}: {cnt}")


def browse(data, tag_filter=None):
    """Interactive browser for extracted positions.

    Commands:
        Enter  = next example
        p      = previous example
        q      = quit
        NUMBER = jump to that index
        t TAG  = re-filter by tag (e.g. 't pin')
    """
    if tag_filter:
        data = [r for r in data if tag_filter in r["tags"]]
        print(f"Filtered to {len(data)} examples with tag '{tag_filter}'")

    if not data:
        print("No examples to show.")
        return

    idx = 0
    while True:
        show_full(data[idx], idx, len(data))
        try:
            cmd = input(f"\n[{idx}/{len(data)-1}] Enter=next  p=prev  q=quit  NUMBER=jump  t TAG=filter: ").strip()
        except (EOFError, KeyboardInterrupt):
            break

        if cmd == "q":
            break
        elif cmd == "p":
            idx = max(0, idx - 1)
        elif cmd == "":
            idx = min(len(data) - 1, idx + 1)
            if idx == len(data) - 1:
                print("(last example)")
        elif cmd.startswith("t "):
            new_tag = cmd[2:].strip()
            browse(rows, tag_filter=new_tag)
            return
        elif cmd.isdigit():
            idx = min(int(cmd), len(data) - 1)
        else:
            print(f"Unknown command: {cmd}")

print("Browser ready. Run: browse(rows)  or  browse(rows, tag_filter='pin')")

In [20]:
# Run this cell to start browsing (uncomment one):
# browse(rows)                            # all examples
# browse(rows, tag_filter="en_passant")   # en passant
# browse(rows, tag_filter="check")        # single check evasion
# browse(rows, tag_filter="double_check") # double check
# browse(rows, tag_filter="illegal_king") # king to attacked sq / illegal castling
# browse(rows, tag_filter="pin")          # pin
# browse(rows, tag_filter="promotion")    # promotion
# browse(rows, tag_filter="vanilla")      # vanilla (no special tag)